# Building an Image

### Introduction

So far we have downloaded images from Dockerhub, and we have used those images to run containers.  But to be a really be able to use Docker, we need the ability to create an image ourselves.  Doing so will also allow us to better understand the other images that we use.

### DockerFile Framework

Docker images are constructed according to a Dockerfile.  Let's see an example by looking at part of our [Ghost DockerFile](https://github.com/docker-library/ghost/blob/2a72c03e339bda5051b37edd0c553fe909e8408d/2/debian/Dockerfile).

```Dockerfile
# Dockerfile
FROM node:10-slim

ENV GOSU_VERSION 1.10
RUN set -x 
...

EXPOSE 2368
CMD ["node", "current/index.js"]
```

At this point, we don't need to understand each of the lines above. For now, it's important to see that our Dockerfile follows a story.  It starts with previous images, builds on it, and then run some task.  This is the format all of our DockerFiles follow:

```
1. FROM base_image
2. RUN build onto environment
3. CMD to run this task
```

So the **FROM** starts from a previous image, the **RUN** is what we add to the environment, and the **CMD** is the task (or process)that is executed when we use an image to start up a new container.  

> In the above Dockerfile, we start the new container by calling `CMD [node, currrent/index.js]`, which starts up a new server.

### Building an Image

Ok, now let's try to use Docker to build an environment that (1) runs a Jupyer notebook and (2) installs software to download kaggle data.  To do so, we'll:

1. Start with a Jupyter notebook, and then
2. Add in the kaggle package.  And then
3. Start up the server

Let's do this via a Dockerfile.

First, let's make a folder called `jupyter-kaggle` and add a new Dockerfile to it.  

> From the terminal, do the following:

```bash
mkdir jupyter-kaggle
touch Dockerfile
```
> Notice that **there is no extension** at the end of our Dockerfile (like `.js` or `.doc`).

Next, let's begin to fill in our Dockerfile, by following our format:

1. FROM base_image
2. RUN build onto environment
3. CMD to run this task

Ok, so now let's just fill in the file.  The first and last steps are often easy to fill in.

```DockerFile
# 1. FROM base_image
FROM jupyter/scipy-notebook

# 2. RUN build onto environment

# 3. CMD to run this task
CMD ["jupyter", "notebook"]
```

To install the kaggle package into our jupyter notebook, we can execute `conda install kaggle` in command line.  Translating this into a Dockerfile looks like the following.  

```Dockerfile
# Dockerfile
# 1. FROM base_image
FROM jupyter/scipy-notebook

# 2. RUN build onto environment
RUN conda install 'kaggle'

# 3. CMD to run this task
CMD ["jupyter", "notebook"]
```

> We can see a similar pattern if we look at the [jupyter/scipy-notebook Dockerfile](https://hub.docker.com/r/jupyter/scipy-notebook/dockerfile) file.

### Building the Image

Now that we have written our Dockerfile, it's time to build our image.  Back in our terminal, from the same folder of our `Dockerfile`, we run the following:

```bash
docker build .
```
Upon doing so, we'll see something like the following: 

```
Sending build context to Docker daemon  2.048kB
Step 1/3 : FROM jupyter/scipy-notebook
 ---> 844815ed865e
Step 2/3 : RUN conda install 'kaggle'
 ---> Running in 4f5b1210656d
Collecting package metadata (current_repodata.json): ...working... done
Solving environment: ...working... done

...

Removing intermediate container 4f5b1210656d
 ---> 29886ca05035
Step 3/3 : CMD ["jupyter", "notebook"]
 ---> Running in 5b0a8c54d511
Removing intermediate container 5b0a8c54d511
 ---> 09c2125c0a49
Successfully built 09c2125c0a49
```

The last line is the `id` of the new image that is built.  Now to have something easier to remember than the id, let's give our image a name by building and tagging our image with the following:

`docker build -t jeffkatzy/jupyter-kaggle .`

```
Sending build context to Docker daemon  2.048kB
Step 1/3 : FROM jupyter/scipy-notebook
 ---> 844815ed865e
Step 2/3 : RUN conda install 'kaggle'
 ---> Using cache
 ---> 29886ca05035
Step 3/3 : CMD ["jupyter", "notebook"]
 ---> Using cache
 ---> 09c2125c0a49
Successfully built 09c2125c0a49
Successfully tagged jeffkatzy/jupyter-kaggle:latest
```

### Viewing our Image

We can run our new image, just like others with the following: 

`docker run -p 8899:8888 jupyter-kaggle`

### Summary

In this lesson, we saw how to build a docker image.  We build a docker image with a `Dockerfile` that follows the structure of: 

```python 
# 1. FROM base_image
# 2. RUN build onto environment
# 3. CMD to run this task
```

Following that format, our Dockerfile looked like the following:

```Dockerfile
# jupyter-kaggle/Dockerfile
# 1. FROM base_image
FROM jupyter/scipy-notebook

# 2. RUN build onto environment
RUN conda install 'kaggle'

# 3. CMD to run this task
CMD ["jupyter", "notebook"]
```

We then built the image with the command: 

`docker build .`

### Resources

[Tiny Containers](https://blog.iron.io/microcontainers-tiny-portable-containers/)

[Docker Python](https://blog.realkinetic.com/building-minimal-docker-containers-for-python-applications-37d0272c52f3?gi=1a4e7a42778e)

[With Authentication](https://github.com/Paperspace/jupyter-docker-stacks/tree/master/scipy-notebook)

[DockerFile Documentation](https://docs.docker.com/engine/reference/builder/#understand-how-cmd-and-entrypoint-interact)